In [9]:
from random import random, randint
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
import math

import numpy.random


class Robot:
    def __init__(self, π_zc = 1, π_zm = 1, π_wm = 1, k = 0.3, speed = 0.01, static_c = False, gradient = False, periodic = False, static_m = False):
        self.sensor_colour = 0 # sensor colour
        self.static_sensor_colour = static_c # bool to check if sensor is static
        self.gradient_sensor_colour = gradient # bool for gradient
        self.periodic_sensor_colour = periodic
        self.sensor_motor = 0 # sensor motor
        self.static_sensor_motor = static_m

        self.belief_colour = 0 # μ colour
        self.belief_motor = 0 # μ motor

        self.pi_z_colour = π_zc # inverse of variance - whatever
        self.pi_z_motor = π_zm # inverse of variance - whatever
        self.pi_w_motor = π_wm # inverse of variance - whatever

        self.k = k # factor k
        self.a = 0 # speed
        self.speed = speed # speed scaling factor

        self.position = 0 # position of robot
        self.free_energy = 0 # F

    '''
    Main function to simulate one step per call
    '''
    # -- Do one step -- #
    def step(self):
        self.update()
        self.move()
        self.log_data()

    '''
    Updating all values
    '''
    # -- Update step -- #
    def update(self):
        self.update_sensor()
        self.update_belief()
        self.update_a()
        self.update_F()
    # -- Update sensor reading, beliefs, a (speed) and F (free energy) -- #
    def update_sensor(self):
        if self.static_sensor_colour:
            self.sensor_colour = 1
        elif self.gradient_sensor_colour:
            self.sensor_colour = 10 - self.position
        elif self.periodic_sensor_colour:
            self.sensor_colour = math.cos(self.position) + 1.1
        else:
            self.sensor_colour = self.belief_colour + numpy.random.normal(0, 1)

        if self.static_sensor_motor:
            self.sensor_motor = 1
        else:
            self.sensor_motor = self.belief_motor + numpy.random.normal(0, 1)
    def update_belief(self):
        self.belief_colour += (-self.k) * ((self.pi_z_colour * (self.belief_colour - self.sensor_colour)) + (self.pi_w_motor * (self.belief_colour - self.belief_motor)))
        self.belief_motor += (-self.k) * ((self.pi_z_motor * (self.belief_motor - self.sensor_motor)) + (self.pi_w_motor * (self.belief_motor - self.belief_colour)))
    def update_a(self):
        self.a += (-self.k) * (self.pi_z_motor * (self.sensor_motor - self.belief_motor))
    def update_F(self):
        self.free_energy = 1/2 * (self.pi_z_motor * math.pow((self.sensor_colour - self.belief_colour), 2) + self.pi_z_motor * math.pow((self.sensor_motor - self.belief_motor), 2) + self.pi_w_motor * math.pow((self.belief_motor - self.belief_colour), 2))

    '''
    Move - just a update position function
    '''
    def move(self):
        self.position += self.speed * self.a

    '''
    Log Position, F, Sensor colour and belief motor
    '''
    def log_data(self):
        print(f"Robot is at position: {self.position} with free energy: {self.free_energy} s_c: {self.belief_colour} and μ_m: {self.belief_motor}")



In [23]:
robot = Robot(static_c=True)
for E in range(6000):
    robot.step()

Robot is at position: 0.013413831796993072 with free energy: 0.7663790898110374 s_c: 1.0 and μ_m: 0.5123097432867041
Robot is at position: 0.02850011511317332 with free energy: 0.2602740189292842 s_c: 0.8536929229860113 and μ_m: 0.41969517470252493
Robot is at position: 0.04516257983413954 with free energy: 0.25489804790399734 s_c: 0.767385721605162 and μ_m: 0.3435366369770876
Robot is at position: 0.062383528799115406 with free energy: 0.10119187619379877 s_c: 0.7100152797351911 and μ_m: 0.4208154490148972
Robot is at position: 0.08211037195103074 with free energy: 0.5278002125755541 s_c: 0.7102507465985456 and μ_m: 0.18687426413082092
Robot is at position: 0.10551601977984872 with free energy: 1.1243798140008616 s_c: 0.6401625778786645 and μ_m: -0.14440284096333578
Robot is at position: 0.12706341649869543 with free energy: 0.31660314665774336 s_c: 0.512744178862465 and μ_m: 0.40269604038647144
Robot is at position: 0.15262731030243704 with free energy: 1.212143801500303 s_c: 0.62590